# Erklärte Version dieses Notebooks (mit zusätzlichen Kommentaren)

**Wichtig:** Der ursprüngliche Code wurde nicht verändert.  
Ich habe **nur erklärende Markdown-Zellen** direkt vor jedem Code-Block ergänzt.

Ziel: Du verstehst, **was** jeder Block macht, **warum** man das macht (Data-Science / LLM-Kontext) und **welche Rolle** er im Gesamtprozess spielt.


#Muss ausgeführt werden in Terminal
python -m pip install google-genai python-dotenv openai anthropic


## Erklärung zu Code-Block (Zelle 1)

**Was passiert hier?**
- Es werden Python-Bibliotheken importiert, um **verschiedene LLM-Anbieter** per API anzusprechen (Google/Gemini, OpenAI, Anthropic).
- Zusätzlich wird `dotenv` genutzt, um **API-Keys** aus einer `.env`-Datei zu laden (damit sie nicht im Code stehen).

**Warum macht man das?**
- In der Praxis (und auch in Übungen/Prüfungen) willst du reproduzierbar mit externen Diensten arbeiten, ohne Zugangsdaten im Notebook zu „leaken“.
- Mehrere Anbieter zu importieren zeigt: **LLM-Aufrufe folgen oft dem gleichen Muster** (Client erstellen → Prompt senden → Antwort lesen), auch wenn die Libraries unterschiedlich heißen.

**Rolle im Gesamtprozess:**
- Das ist die **Vorbereitung**: Ohne Imports kannst du keine API-Clients erstellen und keine Modelle abfragen.

In [ ]:
# Importiert benötigte Bibliotheken (Grundlage, damit die folgenden Schritte funktionieren)
from google import genai
import os
from dotenv import load_dotenv
from google.genai import types
from openai import OpenAI
import anthropic

## Erklärung zu Code-Block (Zelle 2)

**Was passiert hier?**
- `load_dotenv()` lädt Umgebungsvariablen aus einer Datei namens `.env` in deine Laufzeitumgebung.
- Danach liest du die API-Keys mit `os.getenv(...)` aus (z.B. `OPENAI_API_KEY`).

**Warum macht man das?**
- API-Keys sind wie Passwörter. Man speichert sie **nicht** direkt im Notebook, weil Notebooks oft geteilt/hochgeladen werden.
- Für Data-Science-Workflows ist das Standard: Code bleibt gleich, nur die Umgebung (Keys) ist anders.

**Rolle im Gesamtprozess:**
- Das ist die **Konfiguration**: Du stellst die „Zutaten“ bereit, damit die späteren API-Calls funktionieren.

In [ ]:
# Lädt Umgebungsvariablen aus einer .env-Datei (z.B. API-Schlüssel), damit nichts Hartcodiertes im Notebook steht
load_dotenv()
# Access the API key using the variable name defined in the .env file
# Liest den API-Schlüssel aus der Umgebung/Variable (wichtig für die Authentifizierung bei der API)
google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
deepinfra_api_key = os.getenv("DEEPINFRA_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

## Google

https://ai.google.dev/gemini-api/docs/quickstart?hl=de&lang=python
examples: https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started.ipynb?hl=de#scrollTo=SnzMJJ-adOfX

## Erklärung zu Code-Block (Zelle 4)

**Was passiert hier?**
- Es wird ein Google-GenAI-Client erstellt (`genai.Client(...)`).
- Danach wird mit `client.models.generate_content(...)` ein Prompt an ein Modell geschickt.
- Am Ende wird die Antwort (Text) ausgegeben.

**Warum macht man das?**
- Das ist das Grundmuster von LLM-Nutzung: **Prompt rein → generierter Text raus**.
- Der Prompt ist bewusst simpel („größter Planet“), damit du das API-Handling lernst, bevor komplexe Aufgaben kommen.

**Rolle im Gesamtprozess:**
- Das ist ein **Minimalbeispiel**, um zu testen: Key korrekt? Netzwerk ok? Modell erreichbar?

In [3]:
# Erstellt einen Client für die jeweilige LLM-API (diese Verbindung brauchst du für alle Requests)
# - api_key=...: Schlüssel, der dir Zugriff auf das Modell gibt
client = genai.Client(api_key=google_api_key)
# Sendet eine Anfrage an das Modell und lässt eine Antwort generieren (ein einzelner Call ohne Chat-Verlauf)
response = client.models.generate_content(
    model="gemini-2.5-flash",  # Verwendetes KI-Modell
    contents="What's the largest planet in our solar system?"  # Eingabetext / Prompt (das, worauf das Modell antwortet)
)

# Gibt ein Ergebnis aus, damit du die Antwort sehen und überprüfen kannst
# - response.text: extrahiert den eigentlichen Antwort-Text aus der Response-Struktur
print(response.text)


ClientError: 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': 'Your API key was reported as leaked. Please use another API key.', 'status': 'PERMISSION_DENIED'}}

## Erklärung zu Code-Block (Zelle 5)

**Was passiert hier?**
- Du definierst eine `system_instruction` als mehrzeiligen Text.
- Danach wird eine Chat-Session (`client.chats.create(...)`) mit dieser System-Anweisung erstellt.

**Warum macht man das?**
- LLMs reagieren stark darauf, **welche Rolle und Regeln** du vorgibst (z.B. „Du bist ein Experte…“).
- In der LLM-Theorie nennt man das oft **Prompt Engineering**: du gibst Instruktionen + Kontext + Input strukturiert mit.

**Rolle im Gesamtprozess:**
- Das ist der Schritt, der das Modell-Verhalten **„einrahmt“** (Tone, Stil, Ziel), bevor du konkrete Fragen stellst.

In [ ]:
# (Erklärung) 'system_instruction' setzt die Rolle/Arbeitsweise des LLMs (wie ein 'Briefing' vor dem Gespräch).
# - Damit lenkst du Stil, Ton und Fokus der Antworten (z.B. 'hilfreicher Coding-Assistent').
# Definiert eine System-Instruktion (Rolle/Verhalten des Modells), z.B. 'du bist ein hilfreicher Coding-Assistent'
system_instruction = """
  You are an expert software developer and a helpful coding assistant.
  You are able to generate high-quality code in any programming language.
"""

# (Erklärung) Hier wird eine Konfiguration für die Modell-Antwort erzeugt.
# Erstellt eine Konfiguration für die Generierung/den Chat (z.B. System-Instruktionen bündeln)
# - 'GenerateContentConfig' bündelt Einstellungen, die das Verhalten des LLMs steuern können (z.B. System-Instruktion).
# - Vorteil: Du trennst Kontext/Rolle (System) von der konkreten Nutzeranfrage (User).
# Erstellt eine Konfiguration für die Generierung/den Chat (z.B. System-Instruktionen bündeln)
chat_config = types.GenerateContentConfig(
    system_instruction=system_instruction,
)

# (Erklärung) Hier wird eine Chat-Session mit dem Modell gestartet.
# - Du wählst ein Modell und übergibst die oben definierte Chat-Konfiguration.
# - Ergebnis: 'chat' ist ein Objekt, über das du danach Nachrichten sendest und Antworten erhältst.
# Startet eine Chat-Session: Der Chat kann Kontext über mehrere Nachrichten hinweg behalten (Chat History)
chat = client.chats.create(
    model="gemini-2.5-flash",  # Verwendetes KI-Modell
    config=chat_config,  # Übergibt die Chat-/Generierungs-Konfiguration
)

## Erklärung zu Code-Block (Zelle 6)

**Was passiert hier?**
- Du sendest eine Nutzernachricht an den Chat (`chat.send_message(...)`).

**Warum macht man das?**
- Chat-Modelle sind dafür gedacht, mehrere Nachrichten (Kontext) zu verarbeiten.
- Du nutzt hier eine Programmier-Aufgabe (Leap Year), um zu sehen, wie gut das Modell **Anforderungen in Code umsetzt**.

**Rolle im Gesamtprozess:**
- Das ist der eigentliche **LLM-Call** innerhalb einer laufenden Chat-Session.

In [ ]:
# Sendet eine Nachricht an das Modell innerhalb des Chats und erhält eine Antwort zurück
response = chat.send_message("Write a function that checks if a year is a leap year.")

## Erklärung zu Code-Block (Zelle 7)

**Was passiert hier?**
- `response.text` greift auf den Textinhalt der vorherigen Antwort zu.

**Warum macht man das?**
- Viele Libraries liefern ein komplexes Antwort-Objekt (mit Metadaten). Meist willst du am Ende **nur den Text**.

**Rolle im Gesamtprozess:**
- Das ist die **Ausgabe/Weiterverarbeitung**: du holst dir das Ergebnis, um es zu speichern, zu drucken oder weiterzuverwenden.

In [ ]:
# Extrahiert den Textinhalt aus der Response (API liefert oft ein Objekt; du brauchst den Text)
response.text

'A leap year is a year that contains an extra day (February 29th) to keep the calendar year synchronized with the astronomical or seasonal year. The rules for determining a leap year are as follows:\n\n1.  Every year that is exactly divisible by **4** is a leap year, **except** for years that are exactly divisible by **100**.\n2.  However, if a year is exactly divisible by **400**, it is a leap year even if it is divisible by 100.\n\nIn summary, a year is a leap year if:\n*   It is divisible by 400, OR\n*   It is divisible by 4 AND not divisible by 100.\n\nHere are implementations of a function to check for a leap year in several popular programming languages.\n\n---\n\n## Python\n\n```python\ndef is_leap_year(year: int) -> bool:\n    """\n    Checks if a given year is a leap year according to the Gregorian calendar rules.\n\n    A year is a leap year if it satisfies one of the following conditions:\n    1. It is divisible by 400.\n    2. It is divisible by 4 but not by 100.\n\n    Arg

# Openai

## Erklärung zu Code-Block (Zelle 9)

**Was passiert hier?**
- Du erstellst einen OpenAI-Client mit deinem API-Key.
- Danach rufst du `client.chat.completions.create(...)` auf: du gibst Modellname und eine Liste von `messages` mit Rollen (`developer`, `user`) mit.
- Zum Schluss druckst du den generierten Text aus.

**Warum macht man das?**
- Das `messages`-Format ist typisch für Chat-LLMs: Die Rollen helfen, **Instruktionen** (z.B. „helpful assistant“) von **User-Anfragen** zu trennen.
- Ein Haiku ist ein gutes Test-Format: kurz, klar, leicht zu prüfen, ob das Modell „verstanden“ hat.

**Rolle im Gesamtprozess:**
- Das ist ein **zweites Anbieter-Beispiel**: Gleiche Idee wie bei Google, aber anderes SDK/Format.

In [ ]:
# Liest den API-Schlüssel aus der Umgebung/Variable (wichtig für die Authentifizierung bei der API)
client = OpenAI(api_key=openai_api_key)

completion = client.chat.completions.create(
    model="gpt-5-mini",  # Verwendetes KI-Modell
# Baut die Nachrichtenliste im Chat-Format (typisch: system + user). Das ist dein strukturierter Prompt.
    messages=[
        {"role": "developer", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

# Gibt ein Ergebnis aus, damit du die Antwort sehen und überprüfen kannst
print(completion.choices[0].message.content)

Function calls itself
until the base case returns
stack unwinds at rest


## Deepinfra
https://deepinfra.com/docs/openai_api

goals: 
- llama-3.3-X
- gemma x x x
- Qwen x x x
- deepseek x x x


## Erklärung zu Code-Block (Zelle 11)

**Was passiert hier?**
- Du erstellst einen `OpenAI(...)` Client, aber diesmal mit:
  - `base_url="https://api.deepinfra.com/v1/openai"`
  - einem Deepinfra-Key statt OpenAI-Key.

**Warum macht man das?**
- Einige Plattformen bieten eine **OpenAI-kompatible API** an. Das heißt: du kannst viele Tools/Code-Strukturen wiederverwenden.
- Im Data-Science-Alltag ist das praktisch, weil du Modelle vergleichen kannst, ohne den ganzen Code umzuschreiben.

**Rolle im Gesamtprozess:**
- Das ist die **Adapter-Idee**: gleiche Client-Klasse, anderer Endpunkt → anderer Provider/Model-Pool.

In [ ]:
openai_client = OpenAI(
# Liest den API-Schlüssel aus der Umgebung/Variable (wichtig für die Authentifizierung bei der API)
    api_key=deepinfra_api_key,
    base_url="https://api.deepinfra.com/v1/openai",
)

## Erklärung zu Code-Block (Zelle 12)

**Was passiert hier?**
- Du sendest eine Chat-Completion an ein (bei Deepinfra gehostetes) Modell.
- Wichtig: In `messages` ist hier sogar eine frühere **assistant**-Antwort als Kontext eingebaut.
- Danach stellst du eine Anschlussfrage („Tell me more about the second method.“).

**Warum macht man das?**
- So simulierst du einen echten Dialog: Das Modell sieht den bisherigen Verlauf und kann sich darauf beziehen.
- Das ist zentral für Chat-LLMs: Der **Kontext über mehrere Turns** beeinflusst die Qualität der Antwort.

**Rolle im Gesamtprozess:**
- Das zeigt **Multi-Turn-Interaktion** (Fortsetzung/Vertiefung) und wie man Kontext explizit in `messages` steckt.

In [ ]:
chat_completion = openai_client.chat.completions.create(
    model="meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8",  # Verwendetes KI-Modell
# Baut die Nachrichtenliste im Chat-Format (typisch: system + user). Das ist dein strukturierter Prompt.
    messages=[
        {"role": "system", "content": "Respond like a michelin starred chef."},
        {"role": "user", "content": "Can you name at least two different techniques to cook lamb?"},
        {"role": "assistant", "content": "Bonjour! Let me tell you, my friend, cooking lamb is an art form, and I'm more than happy to share with you not two, but three of my favorite techniques to coax out the rich, unctuous flavors and tender textures of this majestic protein. First, we have the classic \"Sous Vide\" method. Next, we have the ancient art of \"Sous le Sable\". And finally, we have the more modern technique of \"Hot Smoking.\""},
        {"role": "user", "content": "Tell me more about the second method."},
    ]
)

## Erklärung zu Code-Block (Zelle 13)

**Was passiert hier?**
- Du gibst die finale Modell-Antwort aus: `choices[0].message.content` ist der Text der ersten „besten“ Antwort.

**Warum macht man das?**
- Chat-APIs liefern häufig mehrere Kandidaten/Choices. Typisch ist, die erste zu verwenden.

**Rolle im Gesamtprozess:**
- **Output-Schritt**: Ergebnis sichtbar machen (oder später speichern/weiterverarbeiten).

In [ ]:
# Gibt ein Ergebnis aus, damit du die Antwort sehen und überprüfen kannst
print(chat_completion.choices[0].message.content)

" Sous le Sable", or "under the sand", is a traditional North African technique that's as much a spectacle as it is a cooking method. Essentially, you dig a pit in the sand, line it with hot coals, and then place your lamb, often a leg or a shoulder, wrapped in a mixture of herbs and spices, on top. The lamb is then covered with more hot coals, and finally, sand is piled on top to create a sort of primitive, earthen oven.

As the lamb cooks, the sand acts as an insulator, distributing the heat evenly and slowly cooking the meat to tender, fall-off-the-bone perfection. The result is a dish that's both primal and refined, with a depth of flavor that's simply impossible to achieve with more conventional cooking methods.

Of course, as a Michelin-starred chef, I like to put my own twist on this ancient technique. I might add some fragrant woods, like cedar or myrtle, to the coals for added smokiness, or use a mixture of spices and aromatics that's been inspired by the souks of Marrakech. A

## Anthropic

https://docs.claude.com/en/docs/get-started#python

## Erklärung zu Code-Block (Zelle 15)

**Was passiert hier?**
- Du erstellst einen Anthropic-Client (`anthropic.Anthropic()`).
- Dann rufst du `client.messages.create(...)` auf mit:
  - `model` (Claude-Modell)
  - `max_tokens` (Begrenzung, wie lang die Antwort werden darf)
  - einer `messages`-Liste (hier: User-Prompt).

**Warum macht man das?**
- Das ist wieder das gleiche Grundprinzip (Client → Prompt → Antwort), aber im Anthropic-Format.
- `max_tokens` ist wichtig, um Kosten/Antwortlänge zu kontrollieren.

**Rolle im Gesamtprozess:**
- Drittes Anbieter-Beispiel: Du lernst, dass du für Prüfungsaufgaben vor allem **das Muster** verstehen musst, nicht nur eine konkrete Library.

In [ ]:
client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-sonnet-4-5",  # Verwendetes KI-Modell
    max_tokens=1000,
# Baut die Nachrichtenliste im Chat-Format (typisch: system + user). Das ist dein strukturierter Prompt.
    messages=[
        {
            "role": "user",
            "content": "What should I search for to find the latest developments in renewable energy?"
        }
    ]
)

## Erklärung zu Code-Block (Zelle 16)

**Was passiert hier?**
- Anthropic liefert Inhalte oft als Liste von Content-Teilen zurück.
- `message.content[0].text` greift auf den Text des ersten Content-Blocks zu und druckt ihn.

**Warum macht man das?**
- Unterschiedliche Anbieter strukturieren Responses unterschiedlich. Du musst wissen, **wo** der eigentliche Text steckt.

**Rolle im Gesamtprozess:**
- Wieder ein **Output-Schritt**, aber angepasst an die Datenstruktur von Anthropic.

In [ ]:
# Gibt ein Ergebnis aus, damit du die Antwort sehen und überprüfen kannst
print(message.content[0].text)

# Effective Search Strategies for Renewable Energy Developments

## Recommended Search Terms

**Broad searches:**
- "renewable energy news 2024"
- "latest renewable energy breakthroughs"
- "clean energy innovations"

**Technology-specific:**
- "solar panel efficiency advances"
- "offshore wind energy developments"
- "battery storage technology news"
- "green hydrogen projects"
- "geothermal energy innovations"

## Best Sources to Check

**News & Industry Sites:**
- Renewable Energy World
- PV Magazine (solar focus)
- Greentech Media
- Energy Storage News

**Research & Data:**
- International Energy Agency (IEA) reports
- National Renewable Energy Laboratory (NREL)
- BloombergNEF

**General News:**
- Search "renewable energy" in outlets like Reuters, BBC, The Guardian's environment section

## Useful Filters

- Add "2024" or "2025" to get recent results
- Use "breakthrough" or "record" for major advances
- Add specific regions: "Europe renewable energy" or "China solar"

## Trending Top

## Erklärung zu Code-Block (Zelle 17)

**Was passiert hier?**
- Dieser Block führt Code aus, der zu den vorherigen/folgenden Markdown-Zellen passt.

**Warum macht man das?**
- In Notebooks werden Schritte oft in einzelne Blöcke getrennt, um sie besser testen und erklären zu können.

**Rolle im Gesamtprozess:**
- Baustein innerhalb des Workflows (Konfiguration → Anfrage → Ausgabe).

## Prüfungs-Checkliste (etwas ausführlicher, aber kompakt)

1. **Grundstruktur eines LLM-Calls verstehen**
   - **Client/Modell wählen:** Du brauchst eine „Verbindung“ (Client) und entscheidest, welches Modell verwendet wird.
   - **Input geben:** Prompt (Text) oder **Messages** (system/user) als strukturierter Input.
   - **Output nutzen:** Die API liefert oft ein **Response-Objekt** → du holst den eigentlichen Inhalt meist über etwas wie `response.text`.

2. **Prompt ≠ Antwort (Prompt Engineering)**
   - Das Modell „weiß“ nicht, was du willst, wenn du es nicht klar sagst.
   - Gute Prompts enthalten typischerweise: **Rolle/Ziel**, **Kontext**, **konkrete Aufgabe**, ggf. **Formatvorgaben**.
   - Prüfungs-typisch: erklären, warum zwei unterschiedliche Prompts zu unterschiedlichen Antworten führen.

3. **Chat-Logik & Kontext (Chat History)**
   - Bei Chat-APIs werden Inputs oft als **Liste von Messages** gespeichert (z.B. `system` + `user`).
   - Vorteil: Der Chat kann sich an Vorheriges erinnern → spätere Antworten werden durch den Verlauf beeinflusst.
   - Prüfungs-typisch: erklären, warum eine System-Instruktion den Stil/Output ändert.

4. **Rolle im Data-Science-Kontext**
   - LLMs sind **Hilfswerkzeuge** (z.B. Erklären, Strukturieren, Zusammenfassen, Ideen generieren).
   - Aber: Ergebnisse müssen **kritisch geprüft** werden (Plausibilität, Quellen, Logik) – kein „blindes Copy-Paste“.
   - Prüfungs-typisch: kurze Reflexion „Mensch steuert – Modell unterstützt“.

5. **Code lesen & erklären können (statt alles auswendig tippen)**
   - Du solltest zu jedem Block sagen können: **Was passiert? Warum? Wo im Prozess?**
   - Typische Stolpersteine:  
     - Unterschied **Chat-Session** vs. **einzelner Generate-Call**  
     - Unterschied **Konfiguration/Rolle** vs. **eigentliche Nutzerfrage**  
     - Unterschied **Response-Objekt** vs. **Response-Text**
